<a href="https://colab.research.google.com/github/fahmisyuhada/Klasifikasi-tulisan-tangan-huruf-aksara-sasak-cnn/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/fahmisyuhada/Klasifikasi-tulisan-tangan-huruf-aksara-sasak-cnn.git project
import zipfile

data_set = zipfile.ZipFile('project/character_with_bouding_box.zip', 'r')

data_set.extractall()
data_set.close()
print('dataset unzip successfully')

Cloning into 'project'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 3.97 MiB | 11.25 MiB/s, done.


Archive:  project/character_with_bouding_box.zip
   creating: karakter3/
   creating: karakter3/ba/
  inflating: karakter3/ba/1.jpeg     
  inflating: karakter3/ba/10.jpeg    
  inflating: karakter3/ba/11.jpeg    
  inflating: karakter3/ba/12.jpeg    
  inflating: karakter3/ba/14.jpeg    
  inflating: karakter3/ba/15.jpeg    
  inflating: karakter3/ba/16.jpeg    
  inflating: karakter3/ba/17.jpeg    
  inflating: karakter3/ba/18.jpeg    
 extracting: karakter3/ba/19.jpeg    
  inflating: karakter3/ba/2.jpeg     
  inflating: karakter3/ba/20.jpeg    
 extracting: karakter3/ba/22.jpeg    
 extracting: karakter3/ba/24.jpeg    
  inflating: karakter3/ba/25.jpeg    
  inflating: karakter3/ba/26.jpeg    
  inflating: karakter3/ba/27.jpeg    
  inflating: karakter3/ba/28.jpeg    
  inflating: karakter3/ba/29.jpeg    
  inflating: karakter3/ba/3.jpeg     
  inflating: karakter3/ba/30.jpeg    
  inflating: karakter3/ba/31.jpeg    
  inflating: karakter3/ba/32.jpeg    
  inflating: karakter3/ba/